In [187]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

trainset = datasets.MNIST('TrainSet', download=True, train=True, transform=transform)
valset = datasets.MNIST('ValSet', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

dataiter = iter(trainloader)
images, Y = dataiter.next()
images = images.numpy()
X = np.resize(images,(64,784))
Y = Y.numpy()

In [189]:
newY = np.zeros(shape=(64,10))
for i in range(Y.shape[0]):
    newY[i,Y[i]] = 1


In [194]:
def getSigmoid(T, x):
    numerator = np.exp(np.matmul(x,T))
    denomenator = numerator + 1 
    sig = np.divide(numerator,denomenator)
    return sig

def getLoss(T, Y, x):
    m = np.prod(Y.shape)
    activation = getSigmoid(T,x)
    loss= np.square(np.subtract(activation, Y))
    print(loss)
    loss = np.sum(loss)
    print(loss)
    loss = np.multiply(loss, 1/(2 * m))
    return loss

def getGradient(T, Y, X):
    activation = getSigmoid(T,X)
    z = np.square(np.subtract(activation, Y))
    gradient = np.dot(X.T, z)
    print(z)
    return gradient
    

In [193]:
theta = np.random.rand(784,10)
getGradient(theta, newY, X)

[[8.02473035e-239 7.07266222e-241 2.93121276e-236 3.62408820e-240
  1.03946858e-227 1.37845732e-228 9.86751633e-242 2.71180632e-238
  4.34653002e-248 1.00000000e+000]
 [1.00000000e+000 3.91528401e-225 2.66524971e-227 1.91364287e-235
  2.01602469e-224 9.75980260e-227 2.37523255e-222 1.97023094e-230
  3.61573972e-238 3.41670662e-225]
 [1.77365627e-223 3.44395858e-225 5.99696064e-220 1.00000000e+000
  9.63310145e-225 5.34166058e-221 1.24045328e-226 5.36472229e-224
  3.21139693e-234 3.19595715e-219]
 [2.64941255e-230 5.32809186e-241 2.71100692e-244 1.00000000e+000
  5.98958449e-236 3.78971126e-231 9.51038762e-239 1.28320469e-236
  2.68795740e-246 1.55115858e-229]
 [3.50787670e-259 2.47286101e-249 1.00000000e+000 2.15089674e-264
  7.30917386e-245 6.35430081e-241 6.10748945e-253 2.18833393e-260
  2.85001476e-261 9.28591328e-242]
 [9.76999120e-249 3.47529970e-255 1.00000000e+000 4.02071508e-261
  2.69048652e-243 1.53513084e-241 4.31631041e-241 2.85765066e-257
  2.43258859e-262 2.07128054e-246

array([[-7., -5., -9., ..., -6., -5., -7.],
       [-7., -5., -9., ..., -6., -5., -7.],
       [-7., -5., -9., ..., -6., -5., -7.],
       ...,
       [-7., -5., -9., ..., -6., -5., -7.],
       [-7., -5., -9., ..., -6., -5., -7.],
       [-7., -5., -9., ..., -6., -5., -7.]])

In [57]:
import numpy as np

# Function to train the logistic regression model
def train_logistic_regression(X, y, num_iterations=100, learning_rate=0.01):
    # Initialize the weights with random values
    weights = np.random.randn(785)
    X = np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)
    
    for i in range(num_iterations):
        # Add a bias term to the input
        
        # Compute the dot product of the input and the weights
        z = np.dot(X, weights)
        
        # Apply the sigmoid function to the dot product to get the predicted probability
        y_pred = 1 / (1 + np.exp(-z))
        
        # Compute the error
        error = y_pred - y
        
        # Update the weights using gradient descent
        weights -= learning_rate * np.dot(X.T, error)
        
    return weights

# Function to make predictions with the logistic regression model
def predict(X, weights):
    # Add a bias term to the input
    X = np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)
    
    # Compute the dot product of the input and the weights
    z = np.dot(X, weights)
    
    # Apply the sigmoid function to the dot product to get the predicted probability
    y_pred = 1 / (1 + np.exp(-z))
    
    # Return the predicted digits by taking the argmax of the predicted probabilities
    return y_pred

# Load the training data
X_train = X
y_train = Y

# Train the model
weights = train_logistic_regression(X_train, y_train)

# Load the test data
X_test = ...

# Make predictions on the test data
y_pred = predict(X, weights)
print(y_pred)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [29]:
import numpy as np

def logistic_regression(x, weights):
    
    # Compute the dot product of the input and the weights
    z = np.dot(x, weights)
    
    # Apply the sigmoid function to the dot product to get the predicted probability
    y_pred = 1 / (1 + np.exp(-z))
    
    # Return the predicted digit by taking the argmax of the predicted probabilities
    return np.argmax(y_pred)